In [ ]:
%matplotlib notebook
import mantid.simpleapi as msi
import matplotlib.pyplot as plt
import numpy as np
import os

from mantid import plots

In [ ]:
Datapath = 'data'


In [ ]:
rn_nums = {}
rn_nums['MT'] = [204358]
rn_nums['Data'] = [201200]
Nom_Ei = 120
flout = {'MT': 'MT120', 'Data': 'Data120'}
MDwks = {'MT': 'MT120_MD', 'Data': 'Data120_MD'}

Common Reduction Parameters

In [ ]:
DGSdict={}
DGSdict['UseIncidentEnergyGuess'] = '1'
DGSdict['EnergyTransferRange'] = '-118.5,0.5,118.5' # be sure the center bin is centered on zero
DGSdict['SofPhiEIsDistribution']='0' # keep events (need to then run RebinTo Workspace and ConvertToDistribution
# DGSdict['HardMaskFile']=HardMaskFile
#DGSdict['GroupingFile'] = '/SNS/ARCS/shared/autoreduce/ARCS_2X1_grouping.xml' # choose 2x1 or some other grouping file created by GenerateGroupingSNSInelastic or GenerateGroupingPowder
DGSdict['IncidentBeamNormalisation']='ByCurrent'
DGSdict['UseProcessedDetVan']='1'
DGSdict['UseBoundsForDetVan']='1'
DGSdict['DetectorVanadiumInputWorkspace']='__VAN'
DGSdict['DetVanIntRangeHigh'] = 4.6
DGSdict['DetVanIntRangeLow'] = 4.3
DGSdict['DetVanIntRangeUnits'] = 'Wavelength'
DGSdict['MedianTestLevelsUp'] = '1'
DGSdict['OutputWorkspace'] = '__OWS'


Load and Merge equivalent runs

In [ ]:
# Load  and merge data 
mgstrs={}
mgstrsmon={}
for idx,typ in enumerate(rn_nums.keys()):
    mgstrs[typ] = ''
    mgstrsmon[typ] = ''
    for rn in rn_nums[typ]:
        wksnm = "Ev{}".format(rn)
        msi.LoadNexusMonitors(Filename=os.path.join(Datapath,'mon.h5'), OutputWorkspace=wksnm+'_mon')
        msi.Load(os.path.join(Datapath,'data.h5'),OutputWorkspace=wksnm)
        mgstrs[typ]+=wksnm+','
        mgstrsmon[typ]+=wksnm+'_mon,'
    mgstrs[typ] = mgstrs[typ][:-1]
    mgstrsmon[typ] =mgstrsmon[typ][:-1]
    msi.MergeRuns(OutputWorkspace=typ, InputWorkspaces=mgstrs[typ])
    msi.MergeRuns(OutputWorkspace=typ+'_mon', InputWorkspaces=mgstrsmon[typ])


Get Ei and T0 values for the runs

In [ ]:
Eis={}
t0s={}
mgdwskpcs = ['Data','MT']
for idx,wksp in enumerate(mgdwskpcs):
    mwksp=wksp+'_mon'
    Eis[wksp], t0s[wksp] =  msi.GetEiT0atSNS(MonitorWorkspace=mwksp, IncidentEnergyGuess=str(Nom_Ei))

Load V detector normalization

In [ ]:
msi.LoadNexus(Filename='data/van.nxs', OutputWorkspace='__VAN')

Reduce data to MD

In [ ]:
for wkspc in mgdwskpcs:
    DGSdict['SampleInputWorkspace'] = wkspc
    DGSdict['SampleInputMonitorWorkspace'] = wkspc+'_mon'
    DGSdict['IncidentEnergyGuess'] = Eis[wkspc]
    DGSdict['TimeZeroGuess'] = t0s[wkspc]
    msi.config['default.facility'] = 'SNS'
    msi.DgsReduction(**DGSdict)
    msi.Divide(LHSWorkspace="__OWS", RHSWorkspace="__VAN", OutputWorkspace="__OWS")
    msi.RebinToWorkspace(WorkspaceToRebin="__OWS", WorkspaceToMatch="__OWS", 
                         OutputWorkspace="__OWS", PreserveEvents='0')
    msi.ConvertToDistribution(Workspace="__OWS") # Divide by bin width
    minvals,maxvals = msi.ConvertToMDMinMaxGlobal("__OWS",'|Q|','Direct')
    msi.ConvertToMD('__OWS',QDimensions='|Q|',dEAnalysisMode='Direct',MinValues=minvals,MaxValues=maxvals,
                    OutputWorkspace=MDwks[wkspc])
    
    #msi.SaveMD(InputWorkspace='MD',Filename=flout[wkspc]+'.h5',SaveLogs=True,SaveSample=True)   

In [ ]:
for wkspc in mgdwskpcs:
    msi.BinMD(InputWorkspace=MDwks[wkspc], 
          AlignedDim0='|Q|,0.0,14,500', # min, max number of bins in Q 
          AlignedDim1='DeltaE,-118.75,118.75,475', # min max number of bins in E 
          OutputWorkspace=MDwks[wkspc]+'H')
    msi.SaveMD(InputWorkspace=MDwks[wkspc]+'H',Filename=flout[wkspc]+'.h5',SaveLogs=True,SaveSample=True)

In [ ]:
msi.mtd.getObjectNames()

In [ ]:
h = msi.mtd['Data120_MDH']
hmt = msi.mtd['MT120_MDH']

In [ ]:
%matplotlib inline 

f,ax=plt.subplots(ncols=2,figsize=(9,4),subplot_kw={'projection':'mantid'})
ax[0].pcolor(h,vmax=5e-4)
cax = ax[1].pcolor(hmt,vmax=5e-4)
f.colorbar(cax, ax=ax[1])
f.tight_layout()
f.savefig('plots.png', bbox_inches='tight')
